# 🏗️ Pipeline de Preprocessing Complet

**Module 2 : Preprocessing et Tokenisation - Synthèse Finale**

---

## 🎯 Objectifs de ce Notebook

Ce notebook constitue la **synthèse finale** du Module 2. Vous allez :

- 🔧 **Construire un pipeline complet** intégrant toutes les techniques
- ⚙️ **Créer une classe configurable** et réutilisable
- 📊 **Mesurer l'impact** de chaque étape
- 🎯 **Adapter le pipeline** selon le cas d'usage
- 🚀 **Optimiser les performances** pour la production
- ✅ **Valider la qualité** du preprocessing

## 🗺️ Architecture du Pipeline

```
📝 Texte Brut
     ↓
🧹 1. Nettoyage (casse, ponctuation, URLs, emojis)
     ↓  
🔧 2. Normalisation (accents, espaces, entités)
     ↓
✂️ 3. Tokenisation (découpage en mots)
     ↓
🛑 4. Filtrage (stopwords, longueur minimale)
     ↓
🌱 5. Lemmatisation/Stemming (forme canonique)
     ↓
✅ Texte Prêt pour ML/NLP
```

## 💡 Pourquoi un Pipeline ?

**Avantages :**
- 🔄 **Reproductible** : Même traitement à chaque fois
- ⚙️ **Configurable** : Adaptation selon le contexte
- 📈 **Mesurable** : Métriques à chaque étape
- 🚀 **Optimisé** : Performance pour la production
- 🧪 **Testable** : Validation et debugging facilités

In [ ]:
# Installation et imports
# !pip install nltk spacy regex pandas matplotlib seaborn
# !python -m spacy download fr_core_news_sm

import re
import time
import warnings
from typing import List, Dict, Tuple, Optional, Union
from dataclasses import dataclass, field
from collections import Counter
import json

# Data science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# NLP Libraries
try:
    import nltk
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize
    from nltk.stem import SnowballStemmer
    # Download required NLTK data
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
    NLTK_AVAILABLE = True
except ImportError:
    print("⚠️ NLTK non disponible, certaines fonctionnalités seront limitées")
    NLTK_AVAILABLE = False

try:
    import spacy
    nlp_fr = spacy.load('fr_core_news_sm')
    SPACY_AVAILABLE = True
except (ImportError, OSError):
    print("⚠️ spaCy français non disponible, utilisation du mode fallback")
    SPACY_AVAILABLE = False

# Configuration
plt.style.use('default')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
warnings.filterwarnings('ignore')

print("✅ Configuration terminée !")
print(f"📚 NLTK disponible : {'✅' if NLTK_AVAILABLE else '❌'}")
print(f"⚡ spaCy français disponible : {'✅' if SPACY_AVAILABLE else '❌'}")

---

# 📊 1. Configuration du Pipeline

Créons d'abord une classe de configuration pour paramétrer notre pipeline :

In [ ]:
@dataclass
class PipelineConfig:
    """Configuration du pipeline de preprocessing"""
    
    # === NETTOYAGE ===
    lowercase: bool = True
    remove_punctuation: bool = True
    remove_numbers: bool = False
    remove_urls: bool = True
    remove_emails: bool = True
    remove_mentions: bool = True  # @username
    remove_hashtags: bool = True  # #hashtag
    
    # === EMOJIS ===
    emoji_strategy: str = "remove"  # "remove", "convert", "keep"
    
    # === NORMALISATION ===
    normalize_whitespace: bool = True
    normalize_accents: bool = False  # Garder les accents français par défaut
    normalize_entities: bool = True  # Dates, montants, etc.
    
    # === TOKENISATION ===
    tokenizer: str = "spacy"  # "spacy", "nltk", "regex", "split"
    handle_contractions: bool = True  # j' → j'
    
    # === FILTRAGE ===
    remove_stopwords: bool = True
    custom_stopwords: List[str] = field(default_factory=list)
    min_token_length: int = 2
    max_token_length: int = 50
    
    # === LEMMATISATION/STEMMING ===
    lemmatize: bool = True
    stem: bool = False  # Alternative au lemmatize
    
    # === PERFORMANCE ===
    batch_size: int = 1000
    n_jobs: int = 1  # Pour traitement parallèle futur
    
    # === DEBUG ===
    verbose: bool = False
    save_intermediate: bool = False
    
    def __post_init__(self):
        """Validation de la configuration"""
        if self.lemmatize and self.stem:
            print("⚠️ Lemmatize et stem activés simultanément. Priorité à lemmatize.")
            self.stem = False
        
        if self.tokenizer == "spacy" and not SPACY_AVAILABLE:
            print("⚠️ spaCy non disponible, basculement vers NLTK")
            self.tokenizer = "nltk" if NLTK_AVAILABLE else "regex"
    
    @classmethod
    def for_domain(cls, domain: str) -> 'PipelineConfig':
        """Configurations prédéfinies par domaine"""
        configs = {
            "general": cls(),
            
            "social_media": cls(
                remove_mentions=False,  # @user peut être important
                remove_hashtags=False,  # #topic peut être important
                emoji_strategy="convert",  # Convertir emojis en sentiment
                normalize_entities=True
            ),
            
            "ecommerce": cls(
                remove_numbers=False,  # Prix et quantités importants
                normalize_entities=False,  # Garder les montants
                remove_urls=True,  # Nettoyer les liens
                lemmatize=True
            ),
            
            "news": cls(
                normalize_entities=True,  # Anonymiser dates/lieux
                remove_urls=True,
                lemmatize=True,
                min_token_length=3
            ),
            
            "academic": cls(
                remove_punctuation=False,  # Ponctuation peut être importante
                normalize_accents=False,  # Garder précision
                lemmatize=True,
                min_token_length=3
            ),
            
            "fast": cls(
                tokenizer="split",
                lemmatize=False,
                stem=True,  # Plus rapide que lemmatize
                normalize_entities=False
            )
        }
        
        return configs.get(domain, configs["general"])
    
    def to_dict(self) -> Dict:
        """Export configuration en dictionnaire"""
        return {
            field.name: getattr(self, field.name) 
            for field in self.__dataclass_fields__.values()
        }
    
    def save(self, filepath: str):
        """Sauvegarde la configuration"""
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(self.to_dict(), f, indent=2, ensure_ascii=False)

# Test des configurations
print("🧪 Configurations disponibles :")
for domain in ["general", "social_media", "ecommerce", "news", "fast"]:
    config = PipelineConfig.for_domain(domain)
    print(f"  • {domain:<12} : tokenizer={config.tokenizer}, lemmatize={config.lemmatize}")

---

# 🔧 2. Classes de Preprocessing

Créons les classes pour chaque étape du pipeline :

In [ ]:
class TextCleaner:
    """Nettoyage et normalisation du texte"""
    
    def __init__(self, config: PipelineConfig):
        self.config = config
        
        # Patterns regex pour nettoyage
        self.patterns = {
            'url': r'https?://[^\s<>"\[\]{}|\\^`]+|www\.[^\s<>"\[\]{}|\\^`]+',
            'email': r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',
            'mention': r'@\w+',
            'hashtag': r'#\w+',
            'phone': r'\+?\d{1,3}[-.]?\(?\d{1,3}\)?[-.]?\d{1,4}[-.]?\d{1,4}[-.]?\d{1,9}',
            'date': r'\b\d{1,2}[/.-]\d{1,2}[/.-]\d{2,4}\b|\b\d{1,2}\s+(janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre)\s+\d{4}\b',
            'money': r'\d+[.,]?\d*\s*(?:€|euros?|\$|dollars?)\b|\b(?:€|\$)\d+[.,]?\d*',
            'emoji': r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF\U00002600-\U000026FF\U00002700-\U000027BF]'
        }
        
        # Dictionnaire emoji → sentiment (simplifié)
        self.emoji_sentiment = {
            '😀': ' positif ', '😁': ' positif ', '😂': ' très positif ', '😃': ' positif ',
            '😄': ' positif ', '😅': ' positif ', '😊': ' positif ', '😌': ' positif ',
            '😍': ' très positif ', '😘': ' positif ', '😋': ' positif ', '😎': ' positif ',
            '😢': ' négatif ', '😭': ' très négatif ', '😡': ' très négatif ', '😠': ' négatif ',
            '😒': ' négatif ', '😞': ' négatif ', '😔': ' négatif ', '😟': ' négatif ',
            '👍': ' positif ', '👎': ' négatif ', '❤️': ' très positif ', '💔': ' négatif ',
            '🔥': ' positif ', '💯': ' très positif ', '✨': ' positif ', '⭐': ' positif '
        }
    
    def clean_text(self, text: str) -> str:
        """Applique toutes les étapes de nettoyage"""
        if not isinstance(text, str):
            return ""
        
        result = text
        
        # 1. Gestion des emojis AVANT suppression ponctuation
        if self.config.emoji_strategy == "convert":
            for emoji, sentiment in self.emoji_sentiment.items():
                result = result.replace(emoji, sentiment)
            # Supprimer les emojis restants
            result = re.sub(self.patterns['emoji'], '', result)
        elif self.config.emoji_strategy == "remove":
            result = re.sub(self.patterns['emoji'], '', result)
        # Si "keep", on ne fait rien
        
        # 2. Suppression éléments web et entités
        if self.config.remove_urls:
            result = re.sub(self.patterns['url'], ' URL ', result)
        
        if self.config.remove_emails:
            result = re.sub(self.patterns['email'], ' EMAIL ', result)
        
        if self.config.remove_mentions:
            result = re.sub(self.patterns['mention'], ' MENTION ', result)
        
        if self.config.remove_hashtags:
            result = re.sub(self.patterns['hashtag'], ' HASHTAG ', result)
        
        # 3. Normalisation entités (si activée)
        if self.config.normalize_entities:
            result = re.sub(self.patterns['phone'], ' TELEPHONE ', result)
            result = re.sub(self.patterns['date'], ' DATE ', result, flags=re.IGNORECASE)
            result = re.sub(self.patterns['money'], ' MONTANT ', result, flags=re.IGNORECASE)
        
        # 4. Minuscules
        if self.config.lowercase:
            result = result.lower()
        
        # 5. Suppression ponctuation
        if self.config.remove_punctuation:
            # Garder les apostrophes pour les contractions françaises
            if self.config.handle_contractions:
                result = re.sub(r"[^\w\s'àâäéèêëïîôöùûüÿç-]", ' ', result)
            else:
                result = re.sub(r'[^\w\s]', ' ', result)
        
        # 6. Suppression nombres (si demandé)
        if self.config.remove_numbers:
            result = re.sub(r'\d+', '', result)
        
        # 7. Normalisation accents (si demandé)
        if self.config.normalize_accents:
            import unicodedata
            result = unicodedata.normalize('NFD', result)
            result = ''.join(c for c in result if unicodedata.category(c) != 'Mn')
        
        # 8. Normalisation espaces
        if self.config.normalize_whitespace:
            result = re.sub(r'\s+', ' ', result).strip()
        
        return result

# Test du nettoyeur
config_test = PipelineConfig()
cleaner = TextCleaner(config_test)

print("🧪 Test du nettoyeur de texte :")
print("=" * 50)

texte_test = "RT @user: Super!!! 😍 https://bit.ly/xyz J'adore ce produit, coûte 50€ le 15/03/2024"
print(f"📝 Original : {texte_test}")
print(f"🧹 Nettoyé : {cleaner.clean_text(texte_test)}")

In [ ]:
class TextTokenizer:
    """Tokenisation du texte"""
    
    def __init__(self, config: PipelineConfig):
        self.config = config
        
        # Initialiser le tokenizer selon la config
        if config.tokenizer == "spacy" and SPACY_AVAILABLE:
            self.nlp = nlp_fr
        elif config.tokenizer == "nltk" and NLTK_AVAILABLE:
            self.nltk_available = True
        else:
            # Fallback vers regex
            if config.handle_contractions:
                # Pattern qui gère les contractions françaises
                self.token_pattern = r"\b\w+(?:'\w+)?\b|\b\w+\b"
            else:
                self.token_pattern = r'\w+'
    
    def tokenize(self, text: str) -> List[str]:
        """Tokenise le texte selon la méthode configurée"""
        if not text or not text.strip():
            return []
        
        tokens = []
        
        if self.config.tokenizer == "spacy" and SPACY_AVAILABLE:
            doc = self.nlp(text)
            tokens = [token.text for token in doc if not token.is_space]
        
        elif self.config.tokenizer == "nltk" and NLTK_AVAILABLE:
            tokens = word_tokenize(text, language='french')
        
        elif self.config.tokenizer == "split":
            tokens = text.split()
        
        else:  # regex fallback
            tokens = re.findall(self.token_pattern, text)
        
        return [token for token in tokens if token and token.strip()]

# Test du tokenizer
tokenizer = TextTokenizer(config_test)

print("🧪 Test du tokenizer :")
print("=" * 40)

texte_clean = "j adore ce produit coute montant le date"
tokens = tokenizer.tokenize(texte_clean)
print(f"📝 Texte : {texte_clean}")
print(f"✂️ Tokens : {tokens}")
print(f"📊 Nombre de tokens : {len(tokens)}")

In [ ]:
class TextFilter:
    """Filtrage des tokens (stopwords, longueur, etc.)"""
    
    def __init__(self, config: PipelineConfig):
        self.config = config
        
        # Charger les stopwords français
        self.stopwords = set()
        
        if config.remove_stopwords:
            # Stopwords de base français
            base_stopwords = {
                'le', 'de', 'et', 'à', 'un', 'il', 'être', 'avoir', 'que', 'pour',
                'dans', 'ce', 'son', 'une', 'sur', 'avec', 'ne', 'se', 'pas', 'tout',
                'plus', 'par', 'grand', 'en', 'son', 'que', 'ce', 'lui', 'au', 'du',
                'des', 'la', 'les', 'je', 'tu', 'nous', 'vous', 'ils', 'elles',
                'mon', 'ma', 'mes', 'ton', 'ta', 'tes', 'sa', 'ses', 'notre', 'nos',
                'votre', 'vos', 'leur', 'leurs', 'est', 'sont', 'était', 'étaient',
                'ai', 'as', 'a', 'avons', 'avez', 'ont', 'été', 'faire', 'fait',
                'faire', 'dis', 'dit', 'cette', 'ces', 'ou', 'où', 'mais', 'donc',
                'car', 'si', 'comme', 'quand', 'bien', 'très', 'aussi', 'alors',
                'ici', 'là', 'maintenant', 'aujourd', 'hui', 'demain', 'hier'
            }
            
            # Ajouter NLTK stopwords si disponible
            if NLTK_AVAILABLE:
                try:
                    nltk_stopwords = set(stopwords.words('french'))
                    base_stopwords.update(nltk_stopwords)
                except:
                    pass
            
            self.stopwords = base_stopwords
            
            # Ajouter stopwords personnalisés
            if config.custom_stopwords:
                self.stopwords.update(config.custom_stopwords)
    
    def filter_tokens(self, tokens: List[str]) -> List[str]:
        """Filtre les tokens selon les critères configurés"""
        if not tokens:
            return []
        
        filtered = []
        
        for token in tokens:
            token_clean = token.lower().strip()
            
            # Vérifier longueur
            if len(token_clean) < self.config.min_token_length:
                continue
            
            if len(token_clean) > self.config.max_token_length:
                continue
            
            # Vérifier stopwords
            if self.config.remove_stopwords and token_clean in self.stopwords:
                continue
            
            # Éviter les tokens vides ou seulement ponctuation
            if not re.search(r'\w', token_clean):
                continue
            
            filtered.append(token_clean)
        
        return filtered

# Test du filtre
text_filter = TextFilter(config_test)

print("🧪 Test du filtre :")
print("=" * 40)
print(f"📊 Nombre de stopwords français : {len(text_filter.stopwords)}")
print(f"🔤 Exemples stopwords : {list(text_filter.stopwords)[:10]}")

tokens_test = ['j', 'adore', 'ce', 'produit', 'coute', 'montant', 'le', 'date', 'et', 'url']
tokens_filtered = text_filter.filter_tokens(tokens_test)
print(f"\n📝 Tokens avant : {tokens_test}")
print(f"🛑 Tokens après filtrage : {tokens_filtered}")

In [ ]:
class TextLemmatizer:
    """Lemmatisation et stemming"""
    
    def __init__(self, config: PipelineConfig):
        self.config = config
        
        # Initialiser lemmatizer/stemmer
        if config.lemmatize and SPACY_AVAILABLE:
            self.nlp = nlp_fr
            self.method = "spacy_lemma"
        elif config.stem and NLTK_AVAILABLE:
            self.stemmer = SnowballStemmer('french')
            self.method = "nltk_stem"
        else:
            # Lemmatisation basique avec dictionnaire
            self.basic_lemmas = {
                # Verbes fréquents
                'suis': 'être', 'es': 'être', 'est': 'être', 'sommes': 'être', 'êtes': 'être', 'sont': 'être',
                'ai': 'avoir', 'as': 'avoir', 'a': 'avoir', 'avons': 'avoir', 'avez': 'avoir', 'ont': 'avoir',
                'fais': 'faire', 'fait': 'faire', 'faisons': 'faire', 'faites': 'faire', 'font': 'faire',
                'vais': 'aller', 'vas': 'aller', 'va': 'aller', 'allons': 'aller', 'allez': 'aller', 'vont': 'aller',
                # Pluriels simples
                'chats': 'chat', 'chiens': 'chien', 'voitures': 'voiture', 'maisons': 'maison',
                'produits': 'produit', 'services': 'service', 'clients': 'client',
                # Adjectifs
                'beaux': 'beau', 'belles': 'beau', 'grands': 'grand', 'grandes': 'grand',
                'bons': 'bon', 'bonnes': 'bon', 'nouveaux': 'nouveau', 'nouvelles': 'nouveau'
            }
            self.method = "basic_lemma"
    
    def lemmatize_tokens(self, tokens: List[str]) -> List[str]:
        """Applique lemmatisation ou stemming aux tokens"""
        if not tokens:
            return []
        
        if not (self.config.lemmatize or self.config.stem):
            return tokens
        
        result = []
        
        if self.method == "spacy_lemma":
            # Joindre les tokens pour traitement spaCy
            text = " ".join(tokens)
            doc = self.nlp(text)
            result = [token.lemma_ for token in doc if not token.is_space]
        
        elif self.method == "nltk_stem":
            result = [self.stemmer.stem(token) for token in tokens]
        
        else:  # basic_lemma
            for token in tokens:
                lemma = self.basic_lemmas.get(token.lower(), token)
                result.append(lemma)
        
        return result

# Test du lemmatizer
lemmatizer = TextLemmatizer(config_test)

print("🧪 Test du lemmatizer :")
print("=" * 40)
print(f"🔧 Méthode utilisée : {lemmatizer.method}")

tokens_test = ['adore', 'produit', 'produits', 'voitures', 'suis', 'fait']
tokens_lemmatized = lemmatizer.lemmatize_tokens(tokens_test)
print(f"\n📝 Tokens avant : {tokens_test}")
print(f"🌱 Tokens lemmatisés : {tokens_lemmatized}")

# Comparaison mot par mot
print("\n🔍 Comparaison détaillée :")
for original, lemma in zip(tokens_test, tokens_lemmatized):
    if original != lemma:
        print(f"  • {original} → {lemma}")
    else:
        print(f"  • {original} (inchangé)")

---

# 🏗️ 3. Pipeline Principal

Assemblons maintenant toutes les étapes dans une classe pipeline complète :

In [ ]:
@dataclass
class ProcessingStats:
    """Statistiques du preprocessing"""
    original_text: str = ""
    final_tokens: List[str] = field(default_factory=list)
    processing_time: float = 0.0
    
    # Compteurs par étape
    chars_before: int = 0
    chars_after_cleaning: int = 0
    tokens_after_tokenization: int = 0
    tokens_after_filtering: int = 0
    tokens_after_lemmatization: int = 0
    
    # Métriques calculées
    @property
    def char_reduction_percent(self) -> float:
        if self.chars_before == 0:
            return 0.0
        return round((self.chars_before - self.chars_after_cleaning) / self.chars_before * 100, 1)
    
    @property
    def token_reduction_percent(self) -> float:
        if self.tokens_after_tokenization == 0:
            return 0.0
        return round((self.tokens_after_tokenization - len(self.final_tokens)) / self.tokens_after_tokenization * 100, 1)
    
    def summary(self) -> str:
        return f"""📊 Statistiques de preprocessing :
• Temps de traitement : {self.processing_time:.3f}s
• Réduction caractères : {self.char_reduction_percent}% ({self.chars_before} → {self.chars_after_cleaning})
• Réduction tokens : {self.token_reduction_percent}% ({self.tokens_after_tokenization} → {len(self.final_tokens)})
• Pipeline : {self.tokens_after_tokenization} → {self.tokens_after_filtering} → {self.tokens_after_lemmatization}
• Tokens finaux : {len(self.final_tokens)}"""


class PreprocessingPipeline:
    """Pipeline complet de preprocessing"""
    
    def __init__(self, config: PipelineConfig):
        self.config = config
        
        # Initialiser les composants
        self.cleaner = TextCleaner(config)
        self.tokenizer = TextTokenizer(config)
        self.filter = TextFilter(config)
        self.lemmatizer = TextLemmatizer(config)
        
        # Historique des traitements
        self.processing_history = []
    
    def process_text(self, text: str, return_stats: bool = False) -> Union[List[str], Tuple[List[str], ProcessingStats]]:
        """
        Traite un texte à travers tout le pipeline
        
        Args:
            text: Texte à traiter
            return_stats: Si True, retourne aussi les statistiques
        
        Returns:
            Liste des tokens finaux, optionnellement avec statistiques
        """
        start_time = time.time()
        
        # Initialiser les stats
        stats = ProcessingStats(original_text=text)
        stats.chars_before = len(text)
        
        # Étape 1: Nettoyage
        cleaned_text = self.cleaner.clean_text(text)
        stats.chars_after_cleaning = len(cleaned_text)
        
        if self.config.verbose:
            print(f"🧹 Après nettoyage : {cleaned_text}")
        
        # Étape 2: Tokenisation
        tokens = self.tokenizer.tokenize(cleaned_text)
        stats.tokens_after_tokenization = len(tokens)
        
        if self.config.verbose:
            print(f"✂️ Après tokenisation : {tokens}")
        
        # Étape 3: Filtrage
        filtered_tokens = self.filter.filter_tokens(tokens)
        stats.tokens_after_filtering = len(filtered_tokens)
        
        if self.config.verbose:
            print(f"🛑 Après filtrage : {filtered_tokens}")
        
        # Étape 4: Lemmatisation
        final_tokens = self.lemmatizer.lemmatize_tokens(filtered_tokens)
        stats.tokens_after_lemmatization = len(final_tokens)
        stats.final_tokens = final_tokens
        
        if self.config.verbose:
            print(f"🌱 Après lemmatisation : {final_tokens}")
        
        # Finaliser les stats
        stats.processing_time = time.time() - start_time
        
        # Sauvegarder l'historique si demandé
        if self.config.save_intermediate:
            self.processing_history.append({
                'original': text,
                'cleaned': cleaned_text,
                'tokens': tokens,
                'filtered': filtered_tokens,
                'final': final_tokens,
                'stats': stats
            })
        
        if return_stats:
            return final_tokens, stats
        return final_tokens
    
    def process_batch(self, texts: List[str], show_progress: bool = True) -> List[List[str]]:
        """
        Traite une liste de textes
        """
        results = []
        
        for i, text in enumerate(texts):
            if show_progress and i % 100 == 0:
                print(f"📈 Progression : {i}/{len(texts)} textes traités")
            
            tokens = self.process_text(text)
            results.append(tokens)
        
        if show_progress:
            print(f"✅ Traitement terminé : {len(texts)} textes")
        
        return results
    
    def analyze_vocabulary(self, texts: List[str]) -> Dict:
        """
        Analyse le vocabulaire avant/après preprocessing
        """
        # Vocabulaire avant
        vocab_before = set()
        for text in texts:
            words = text.lower().split()
            vocab_before.update(words)
        
        # Vocabulaire après
        vocab_after = set()
        processed = self.process_batch(texts, show_progress=False)
        for tokens in processed:
            vocab_after.update(tokens)
        
        return {
            'vocab_size_before': len(vocab_before),
            'vocab_size_after': len(vocab_after),
            'reduction_percent': round((len(vocab_before) - len(vocab_after)) / len(vocab_before) * 100, 1),
            'vocab_before': vocab_before,
            'vocab_after': vocab_after
        }
    
    def save_config(self, filepath: str):
        """Sauvegarde la configuration"""
        self.config.save(filepath)
    
    def get_pipeline_info(self) -> str:
        """Retourne info sur la configuration du pipeline"""
        info = f"""🔧 Configuration du Pipeline :
• Tokenizer : {self.config.tokenizer}
• Lemmatisation : {self.config.lemmatize}
• Stemming : {self.config.stem}
• Suppression stopwords : {self.config.remove_stopwords}
• Suppression ponctuation : {self.config.remove_punctuation}
• Gestion emojis : {self.config.emoji_strategy}
• Normalisation entités : {self.config.normalize_entities}
• Longueur min tokens : {self.config.min_token_length}
• Stopwords personnalisés : {len(self.config.custom_stopwords)}"""
        return info

print("✅ Pipeline complet créé !")

---

# 🧪 4. Tests et Démonstrations

Testons notre pipeline sur différents types de textes :

In [ ]:
# Créer pipeline avec config par défaut
config_demo = PipelineConfig(verbose=True)
pipeline = PreprocessingPipeline(config_demo)

print("🚀 DÉMONSTRATION DU PIPELINE COMPLET")
print("=" * 60)
print(pipeline.get_pipeline_info())
print()

# Test sur un texte complexe
texte_complexe = """
Salut @marie ! 😍 J'ai acheté ce super produit sur https://boutique.com pour 199,99€ le 15/03/2024.
Contact : support@boutique.fr ou 01.23.45.67.89. 
C'est vraiment GÉNIAL !!! Je recommande à 100% 👍 #shopping #bonplan
""".strip()

print("📝 TEXTE DE TEST :")
print(f"\n{texte_complexe}\n")

print("🔧 TRAITEMENT ÉTAPE PAR ÉTAPE :")
print("=" * 50)

tokens_finaux, stats = pipeline.process_text(texte_complexe, return_stats=True)

print(f"\n✅ RÉSULTAT FINAL :")
print(f"Tokens : {tokens_finaux}")
print(f"Phrase reconstruite : '{' '.join(tokens_finaux)}'")
print(f"\n{stats.summary()}")

In [ ]:
# Test comparatif entre différentes configurations
textes_test = [
    "RT @user: LOL!!! C'est GÉNIAL 😍😍😍 https://bit.ly/xyz #amazing",
    "J'adore ce produit! Coûte 50€, livraison gratuite. Contact: info@shop.fr",
    "Rendez-vous le 15 mars 2024 à 14h30 pour discuter du budget.",
    "Super service client!!! Ils sont très professionnels et rapides 👍",
    "N'hésitez pas à me contacter au 01.23.45.67.89 pour plus d'infos."
]

# Tester différentes configs
configs_test = {
    "Social Media": PipelineConfig.for_domain("social_media"),
    "E-commerce": PipelineConfig.for_domain("ecommerce"),
    "News": PipelineConfig.for_domain("news"),
    "Fast": PipelineConfig.for_domain("fast")
}

print("⚔️ COMPARAISON DES CONFIGURATIONS")
print("=" * 60)

# Analyser chaque config
resultats_comparaison = []

for nom_config, config in configs_test.items():
    pipeline_test = PreprocessingPipeline(config)
    
    # Analyser le vocabulaire
    vocab_analysis = pipeline_test.analyze_vocabulary(textes_test)
    
    # Mesurer le temps moyen
    start_time = time.time()
    processed = pipeline_test.process_batch(textes_test, show_progress=False)
    avg_time = (time.time() - start_time) / len(textes_test)
    
    # Calculer tokens moyens
    avg_tokens = np.mean([len(tokens) for tokens in processed])
    
    resultats_comparaison.append({
        'Configuration': nom_config,
        'Vocab avant': vocab_analysis['vocab_size_before'],
        'Vocab après': vocab_analysis['vocab_size_after'],
        'Réduction (%)': vocab_analysis['reduction_percent'],
        'Tokens moyens': round(avg_tokens, 1),
        'Temps (ms)': round(avg_time * 1000, 1)
    })
    
    print(f"\n🔧 **{nom_config}** :")
    print(f"  • Réduction vocabulaire : {vocab_analysis['reduction_percent']}%")
    print(f"  • Tokens moyens par texte : {avg_tokens:.1f}")
    print(f"  • Temps moyen : {avg_time*1000:.1f}ms")

# Créer DataFrame pour analyse
df_comparaison = pd.DataFrame(resultats_comparaison)
print(f"\n📊 **TABLEAU RÉCAPITULATIF :**")
print(df_comparaison.to_string(index=False))

---

# 📈 5. Visualisations et Analyses

Créons des visualisations pour analyser l'impact du preprocessing :

In [ ]:
# Visualisation des comparaisons
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Graphique 1: Réduction du vocabulaire
configs = df_comparaison['Configuration']
reductions = df_comparaison['Réduction (%)']
colors = plt.cm.viridis(np.linspace(0, 1, len(configs)))

bars1 = ax1.bar(configs, reductions, color=colors, alpha=0.8)
ax1.set_title('📉 Réduction du Vocabulaire par Configuration', fontsize=12, fontweight='bold')
ax1.set_ylabel('Réduction (%)')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3)

for bar, val in zip(bars1, reductions):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
             f'{val}%', ha='center', va='bottom', fontweight='bold')

# Graphique 2: Taille vocabulaire avant/après
x = np.arange(len(configs))
width = 0.35

ax2.bar(x - width/2, df_comparaison['Vocab avant'], width, 
        label='Avant', color='lightcoral', alpha=0.8)
ax2.bar(x + width/2, df_comparaison['Vocab après'], width,
        label='Après', color='lightblue', alpha=0.8)

ax2.set_title('📊 Taille du Vocabulaire Avant/Après', fontsize=12, fontweight='bold')
ax2.set_ylabel('Nombre de mots uniques')
ax2.set_xticks(x)
ax2.set_xticklabels(configs, rotation=45)
ax2.legend()
ax2.grid(True, alpha=0.3)

# Graphique 3: Nombre moyen de tokens
bars3 = ax3.bar(configs, df_comparaison['Tokens moyens'], 
                color=plt.cm.plasma(np.linspace(0, 1, len(configs))), alpha=0.8)
ax3.set_title('📝 Nombre Moyen de Tokens par Texte', fontsize=12, fontweight='bold')
ax3.set_ylabel('Tokens moyens')
ax3.tick_params(axis='x', rotation=45)
ax3.grid(True, alpha=0.3)

for bar, val in zip(bars3, df_comparaison['Tokens moyens']):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
             f'{val}', ha='center', va='bottom', fontweight='bold')

# Graphique 4: Performance (temps de traitement)
bars4 = ax4.bar(configs, df_comparaison['Temps (ms)'],
                color=plt.cm.cool(np.linspace(0, 1, len(configs))), alpha=0.8)
ax4.set_title('⚡ Performance (Temps de Traitement)', fontsize=12, fontweight='bold')
ax4.set_ylabel('Temps moyen (ms)')
ax4.tick_params(axis='x', rotation=45)
ax4.grid(True, alpha=0.3)

for bar, val in zip(bars4, df_comparaison['Temps (ms)']):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
             f'{val}ms', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Analyse des résultats
print("\n💡 **ANALYSE DES RÉSULTATS :**")
print("=" * 50)

# Configuration la plus efficace
idx_max_reduction = df_comparaison['Réduction (%)'].idxmax()
config_max_reduction = df_comparaison.loc[idx_max_reduction, 'Configuration']
max_reduction = df_comparaison.loc[idx_max_reduction, 'Réduction (%)']

print(f"🏆 **Meilleure réduction vocabulaire** : {config_max_reduction} ({max_reduction}%)")

# Configuration la plus rapide
idx_min_time = df_comparaison['Temps (ms)'].idxmin()
config_min_time = df_comparaison.loc[idx_min_time, 'Configuration']
min_time = df_comparaison.loc[idx_min_time, 'Temps (ms)']

print(f"⚡ **Configuration la plus rapide** : {config_min_time} ({min_time}ms)")

# Recommandations
print(f"\n🎯 **RECOMMANDATIONS :**")
print(f"• Pour **réduction maximale** du vocabulaire → {config_max_reduction}")
print(f"• Pour **performance optimale** → {config_min_time}")
print(f"• Pour **équilibre** qualité/vitesse → News ou E-commerce")
print(f"• Pour **médias sociaux** → Social Media (gère emojis et mentions)")

---

# 🎯 6. Exercices Pratiques

À votre tour ! Expérimentez avec le pipeline :

In [ ]:
# 🎯 EXERCICE 1: Créez votre configuration personnalisée

print("🎯 EXERCICE 1 : Configuration Personnalisée")
print("=" * 50)

# Modifiez ces paramètres selon vos besoins
ma_config = PipelineConfig(
    # À personnaliser :
    remove_punctuation=True,
    emoji_strategy="convert",  # "remove", "convert", "keep"
    remove_stopwords=True,
    lemmatize=True,
    min_token_length=3,
    tokenizer="spacy",  # "spacy", "nltk", "regex", "split"
    normalize_entities=True,
    
    # Stopwords personnalisés pour votre domaine
    custom_stopwords=["super", "vraiment", "très", "assez"],
    
    verbose=True  # Pour voir les étapes
)

mon_pipeline = PreprocessingPipeline(ma_config)

print(mon_pipeline.get_pipeline_info())
print()

# Testez sur vos propres textes
mes_textes = [
    "Remplacez ce texte par vos propres exemples !",
    "Exemple : J'adore vraiment ce super produit 😍 coûte 50€",
    "Autre exemple : Contact@entreprise.fr pour plus d'infos"
    # Ajoutez vos textes ici...
]

print("🧪 **Test de votre configuration :**")
for i, texte in enumerate(mes_textes, 1):
    print(f"\n📝 **Texte {i}** : {texte}")
    tokens, stats = mon_pipeline.process_text(texte, return_stats=True)
    print(f"✅ **Résultat** : {tokens}")
    print(f"📊 **Stats** : {stats.token_reduction_percent}% réduction, {stats.processing_time:.3f}s")

In [ ]:
# 🎯 EXERCICE 2: Analyse de corpus

print("\n🎯 EXERCICE 2 : Analyse d'un Corpus")
print("=" * 50)

# Dataset d'exemple (remplacez par vos données)
corpus_exemple = [
    "Excellent produit ! Je le recommande vivement 😍 Prix : 99€",
    "Service client décevant... Pas de réponse depuis 3 jours 😞",
    "Livraison rapide, emballage soigné. Contact : support@shop.fr",
    "Qualité/prix imbattable ! Disponible sur https://boutique.com",
    "Attention arnaque !!! N'achetez pas ici 😡 #attention",
    "Super expérience d'achat, je recommande à 100% 👍",
    "Produit conforme à la description. Livré le 15/03/2024",
    "SAV très réactif au 01.23.45.67.89. Problème résolu rapidement.",
    "Promo intéressante : -30% jusqu'au 31/12/2024 #bonplan",
    "Article de qualité, mais un peu cher à 150€... Livraison OK"
]

# Analyser avec différentes configurations
configs_analyse = {
    "Basique": PipelineConfig(lemmatize=False, normalize_entities=False),
    "Complet": PipelineConfig(),
    "E-commerce": PipelineConfig.for_domain("ecommerce")
}

resultats_analyse = {}

for nom, config in configs_analyse.items():
    pipeline_analyse = PreprocessingPipeline(config)
    
    # Traitement du corpus
    start_time = time.time()
    tokens_corpus = pipeline_analyse.process_batch(corpus_exemple, show_progress=False)
    temps_total = time.time() - start_time
    
    # Analyse vocabulaire
    vocab_analysis = pipeline_analyse.analyze_vocabulary(corpus_exemple)
    
    # Statistiques
    tous_tokens = [token for tokens in tokens_corpus for token in tokens]
    freq_tokens = Counter(tous_tokens)
    
    resultats_analyse[nom] = {
        'tokens_corpus': tokens_corpus,
        'vocab_analysis': vocab_analysis,
        'temps_total': temps_total,
        'freq_tokens': freq_tokens,
        'tokens_total': len(tous_tokens),
        'tokens_uniques': len(set(tous_tokens))
    }

# Affichage des résultats
print("📊 **RÉSULTATS DE L'ANALYSE :**\n")

for nom, resultats in resultats_analyse.items():
    print(f"🔧 **Configuration {nom}** :")
    print(f"  • Vocabulaire : {resultats['vocab_analysis']['vocab_size_before']} → {resultats['vocab_analysis']['vocab_size_after']} mots ({resultats['vocab_analysis']['reduction_percent']}%)")
    print(f"  • Tokens total : {resultats['tokens_total']}")
    print(f"  • Tokens uniques : {resultats['tokens_uniques']}")
    print(f"  • Temps traitement : {resultats['temps_total']:.3f}s")
    print(f"  • Top 5 mots : {list(resultats['freq_tokens'].most_common(5))}")
    print()

# Comparaison visuelle simple
noms = list(resultats_analyse.keys())
reductions = [resultats_analyse[nom]['vocab_analysis']['reduction_percent'] for nom in noms]
temps = [resultats_analyse[nom]['temps_total'] * 1000 for nom in noms]  # en ms

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Réduction vocabulaire
ax1.bar(noms, reductions, color=['skyblue', 'lightgreen', 'salmon'])
ax1.set_title('📉 Réduction Vocabulaire (%)')
ax1.set_ylabel('Réduction (%)')
for i, v in enumerate(reductions):
    ax1.text(i, v + 1, f'{v}%', ha='center', fontweight='bold')

# Temps de traitement
ax2.bar(noms, temps, color=['lightcoral', 'lightblue', 'lightgreen'])
ax2.set_title('⚡ Temps de Traitement (ms)')
ax2.set_ylabel('Temps (ms)')
for i, v in enumerate(temps):
    ax2.text(i, v + 0.1, f'{v:.1f}ms', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("💡 **OBSERVATIONS :**")
print(f"• Configuration la plus efficace : {noms[reductions.index(max(reductions))]} ({max(reductions)}% réduction)")
print(f"• Configuration la plus rapide : {noms[temps.index(min(temps))]} ({min(temps):.1f}ms)")

In [ ]:
# 🎯 EXERCICE 3: Optimisation de performance

print("\n🎯 EXERCICE 3 : Benchmark de Performance")
print("=" * 50)

# Générer un corpus de test plus grand
def generer_corpus_test(taille: int) -> List[str]:
    """Génère un corpus de test de taille donnée"""
    templates = [
        "Super produit ! Je recommande 😍 Prix : {}€",
        "Service décevant... Contact : {}@email.fr",
        "Livré le {} rapidement. Site : https://{}.com",
        "Attention ! N'achetez pas ici 😡 #arnaque",
        "Expérience parfaite 👍 Tél : 01.{}.{}.{}.{}",
        "Qualité/prix correct. Disponible jusqu'au {}/{}/2024"
    ]
    
    corpus = []
    for i in range(taille):
        template = templates[i % len(templates)]
        if '{}' in template:
            # Remplacer les placeholders
            if 'Prix' in template:
                text = template.format(50 + (i % 100))
            elif '@email' in template:
                text = template.format(f"user{i%100}")
            elif 'https' in template:
                text = template.format(f"site{i%10}")
            elif 'Tél' in template:
                text = template.format(
                    20 + (i%8), 30 + (i%7), 40 + (i%6), 50 + (i%9)
                )
            elif '2024' in template:
                text = template.format((i%28)+1, (i%12)+1)
            else:
                text = template
        else:
            text = template
        corpus.append(text)
    
    return corpus

# Générer différentes tailles de corpus
tailles = [100, 500, 1000]
configs_bench = {
    "Fast": PipelineConfig.for_domain("fast"),
    "Standard": PipelineConfig(),
    "Complet": PipelineConfig(normalize_entities=True, emoji_strategy="convert")
}

resultats_bench = []

print("⏱️ **BENCHMARK EN COURS...**\n")

for taille in tailles:
    corpus_test = generer_corpus_test(taille)
    print(f"📊 Corpus de {taille} textes :")
    
    for nom_config, config in configs_bench.items():
        pipeline_bench = PreprocessingPipeline(config)
        
        # Mesurer le temps
        start_time = time.time()
        resultats = pipeline_bench.process_batch(corpus_test, show_progress=False)
        temps_total = time.time() - start_time
        
        # Calculer métriques
        temps_par_texte = temps_total / len(corpus_test) * 1000  # ms
        tokens_moyens = np.mean([len(tokens) for tokens in resultats])
        
        resultats_bench.append({
            'Taille Corpus': taille,
            'Configuration': nom_config,
            'Temps Total (s)': round(temps_total, 3),
            'Temps/Texte (ms)': round(temps_par_texte, 2),
            'Tokens Moyens': round(tokens_moyens, 1),
            'Vitesse (textes/s)': round(len(corpus_test) / temps_total, 1)
        })
        
        print(f"  • {nom_config:<10} : {temps_total:.3f}s ({temps_par_texte:.2f}ms/texte)")
    
    print()

# Créer DataFrame et visualiser
df_bench = pd.DataFrame(resultats_bench)

print("📊 **RÉSULTATS COMPLETS :**")
print(df_bench.to_string(index=False))

# Graphique de performance
plt.figure(figsize=(12, 8))

# Subplot 1: Temps par texte selon la taille
plt.subplot(2, 2, 1)
for config in configs_bench.keys():
    data = df_bench[df_bench['Configuration'] == config]
    plt.plot(data['Taille Corpus'], data['Temps/Texte (ms)'], 
             marker='o', label=config, linewidth=2)
plt.xlabel('Taille du Corpus')
plt.ylabel('Temps par Texte (ms)')
plt.title('⚡ Performance vs Taille Corpus')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 2: Vitesse de traitement
plt.subplot(2, 2, 2)
for config in configs_bench.keys():
    data = df_bench[df_bench['Configuration'] == config]
    plt.plot(data['Taille Corpus'], data['Vitesse (textes/s)'], 
             marker='s', label=config, linewidth=2)
plt.xlabel('Taille du Corpus')
plt.ylabel('Vitesse (textes/seconde)')
plt.title('🚀 Débit de Traitement')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 3: Comparaison configs pour 1000 textes
plt.subplot(2, 2, 3)
data_1000 = df_bench[df_bench['Taille Corpus'] == 1000]
plt.bar(data_1000['Configuration'], data_1000['Temps/Texte (ms)'],
        color=['red', 'blue', 'green'], alpha=0.7)
plt.ylabel('Temps par Texte (ms)')
plt.title('📊 Performance sur 1000 Textes')
plt.xticks(rotation=45)
for i, v in enumerate(data_1000['Temps/Texte (ms)']):
    plt.text(i, v + 0.1, f'{v}ms', ha='center', fontweight='bold')

# Subplot 4: Tokens moyens par config
plt.subplot(2, 2, 4)
plt.bar(data_1000['Configuration'], data_1000['Tokens Moyens'],
        color=['orange', 'purple', 'brown'], alpha=0.7)
plt.ylabel('Tokens Moyens')
plt.title('📝 Nombre Moyen de Tokens')
plt.xticks(rotation=45)
for i, v in enumerate(data_1000['Tokens Moyens']):
    plt.text(i, v + 0.1, f'{v}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

# Recommandations finales
print("\n🎯 **RECOMMANDATIONS FINALES :**")
config_plus_rapide = data_1000.loc[data_1000['Temps/Texte (ms)'].idxmin(), 'Configuration']
vitesse_max = data_1000['Vitesse (textes/s)'].max()

print(f"• **Configuration la plus rapide** : {config_plus_rapide}")
print(f"• **Débit maximum** : {vitesse_max} textes/seconde")
print(f"• **Pour production** : Privilégier 'Fast' si volume important")
print(f"• **Pour qualité** : Privilégier 'Complet' si précision importante")
print(f"• **Compromis** : 'Standard' pour équilibre qualité/vitesse")

---

# 🔧 7. Sauvegarde et Utilisation en Production

Apprenons à sauvegarder et réutiliser nos pipelines :

In [ ]:
# Sauvegarde de configuration
print("💾 SAUVEGARDE ET CHARGEMENT")
print("=" * 40)

# Créer une config optimisée
config_production = PipelineConfig(
    # Optimisé pour la production
    tokenizer="spacy",
    lemmatize=True,
    remove_stopwords=True,
    normalize_entities=True,
    emoji_strategy="convert",
    min_token_length=3,
    
    # Performance
    batch_size=1000,
    verbose=False,
    
    # Stopwords métier
    custom_stopwords=["produit", "service", "client", "entreprise"]
)

# Sauvegarder
try:
    config_production.save("config_production.json")
    print("✅ Configuration sauvegardée dans 'config_production.json'")
except Exception as e:
    print(f"⚠️ Erreur sauvegarde : {e}")

# Créer le pipeline de production
pipeline_production = PreprocessingPipeline(config_production)

print("\n🏭 **PIPELINE DE PRODUCTION CRÉÉ :**")
print(pipeline_production.get_pipeline_info())

# Fonction utilitaire pour usage simple
def preprocess_text_simple(text: str, domain: str = "general") -> List[str]:
    """
    Fonction simple pour preprocessing rapide
    
    Args:
        text: Texte à traiter
        domain: Domaine d'application ("general", "social_media", "ecommerce", etc.)
    
    Returns:
        Liste de tokens preprocessés
    """
    config = PipelineConfig.for_domain(domain)
    pipeline = PreprocessingPipeline(config)
    return pipeline.process_text(text)

# Test de la fonction simple
print("\n🧪 **TEST FONCTION SIMPLE :**")
texte_test = "J'adore ce produit! 😍 Coûte 50€, contact@shop.fr"

for domain in ["general", "ecommerce", "social_media"]:
    tokens = preprocess_text_simple(texte_test, domain)
    print(f"• {domain:<12} : {tokens}")

print("\n📋 **CLASSE WRAPPER POUR PRODUCTION :**")

In [ ]:
class TextPreprocessor:
    """
    Classe wrapper simplifiée pour utilisation en production
    """
    
    def __init__(self, domain: str = "general", config_path: Optional[str] = None):
        """
        Initialise le preprocessor
        
        Args:
            domain: Domaine prédéfini ou "custom" si config_path fourni
            config_path: Chemin vers fichier config JSON (optionnel)
        """
        if config_path:
            # Charger config depuis fichier
            try:
                with open(config_path, 'r', encoding='utf-8') as f:
                    config_dict = json.load(f)
                # Reconstruire la config (simplifié)
                self.config = PipelineConfig(**config_dict)
            except Exception as e:
                print(f"⚠️ Erreur chargement config : {e}")
                self.config = PipelineConfig.for_domain("general")
        else:
            self.config = PipelineConfig.for_domain(domain)
        
        self.pipeline = PreprocessingPipeline(self.config)
        self.domain = domain
    
    def preprocess(self, text: Union[str, List[str]]) -> Union[List[str], List[List[str]]]:
        """
        Preprocess un texte ou une liste de textes
        
        Args:
            text: Texte(s) à traiter
        
        Returns:
            Tokens ou liste de tokens
        """
        if isinstance(text, str):
            return self.pipeline.process_text(text)
        elif isinstance(text, list):
            return self.pipeline.process_batch(text, show_progress=True)
        else:
            raise ValueError("Input doit être str ou List[str]")
    
    def analyze(self, texts: List[str]) -> Dict:
        """
        Analyse un corpus de textes
        
        Returns:
            Dictionnaire avec statistiques
        """
        return self.pipeline.analyze_vocabulary(texts)
    
    def get_stats(self, text: str) -> ProcessingStats:
        """
        Obtient statistiques détaillées pour un texte
        """
        _, stats = self.pipeline.process_text(text, return_stats=True)
        return stats
    
    def save_config(self, filepath: str):
        """Sauvegarde la configuration actuelle"""
        self.config.save(filepath)
    
    def __repr__(self):
        return f"TextPreprocessor(domain='{self.domain}', tokenizer='{self.config.tokenizer}')"


# Démonstration de la classe wrapper
print("🚀 **DÉMONSTRATION CLASSE WRAPPER :**")
print("=" * 50)

# Créer preprocessors pour différents domaines
preprocessors = {
    "E-commerce": TextPreprocessor("ecommerce"),
    "Social Media": TextPreprocessor("social_media"),
    "Rapide": TextPreprocessor("fast")
}

# Texte de test
texte_demo = "Super expérience ! 😍 J'ai acheté pour 99€ sur https://shop.fr. Contact : info@shop.fr #génial"

print(f"📝 **Texte de test :**\n{texte_demo}\n")

for nom, preprocessor in preprocessors.items():
    print(f"🔧 **{nom}** :")
    
    # Preprocessing simple
    tokens = preprocessor.preprocess(texte_demo)
    print(f"  • Tokens : {tokens}")
    
    # Statistiques
    stats = preprocessor.get_stats(texte_demo)
    print(f"  • Réduction : {stats.token_reduction_percent}%")
    print(f"  • Temps : {stats.processing_time:.3f}s")
    print()

# Test sur plusieurs textes
print("📊 **TEST SUR CORPUS :**")
corpus_demo = [
    "Produit excellent ! Je recommande 😍",
    "Service client décevant... 😞 Contact : help@site.com",
    "Livraison rapide, prix correct : 45€"
]

preprocessor_demo = TextPreprocessor("ecommerce")
resultats_corpus = preprocessor_demo.preprocess(corpus_demo)

print(f"📈 Corpus traité : {len(resultats_corpus)} textes")
for i, (original, tokens) in enumerate(zip(corpus_demo, resultats_corpus)):
    print(f"  {i+1}. {original[:40]}... → {len(tokens)} tokens")

# Analyse du corpus
analyse_corpus = preprocessor_demo.analyze(corpus_demo)
print(f"\n📊 **Analyse vocabulaire :**")
print(f"  • Vocabulaire avant : {analyse_corpus['vocab_size_before']} mots")
print(f"  • Vocabulaire après : {analyse_corpus['vocab_size_after']} mots")
print(f"  • Réduction : {analyse_corpus['reduction_percent']}%")

print("\n✅ **Classe wrapper prête pour la production !**")

---

# 🎉 Conclusion et Synthèse

## 🏆 Ce que Vous Avez Accompli

Félicitations ! Vous avez créé un **pipeline de preprocessing NLP complet et professionnel**. Voici ce que vous maîtrisez maintenant :

### ✅ **Compétences Techniques Acquises**

1. **🧹 Nettoyage de texte** : Gestion de la casse, ponctuation, URLs, emojis
2. **🔧 Normalisation** : Accents, espaces, entités (dates, montants, etc.)
3. **✂️ Tokenisation** : Différentes stratégies (spaCy, NLTK, regex)
4. **🛑 Filtrage** : Stopwords, longueur, tokens spéciaux
5. **🌱 Lemmatisation/Stemming** : Réduction à la forme canonique
6. **📊 Métriques et analyse** : Mesure de l'impact du preprocessing
7. **⚙️ Configuration modulaire** : Adaptation par domaine
8. **🚀 Optimisation** : Performance et scalabilité

### 🎯 **Configurations Maîtrisées**

| **Domaine** | **Cas d'usage** | **Spécificités** |
|---|---|---|
| **General** | Usage polyvalent | Équilibre qualité/vitesse |
| **Social Media** | Tweets, posts | Gère emojis, mentions, hashtags |
| **E-commerce** | Avis clients | Préserve montants, supprime URLs |
| **News** | Articles presse | Anonymise entités, qualité élevée |
| **Fast** | Gros volumes | Performance maximale |

### 📈 **Métriques de Performance**

- **Réduction vocabulaire** : 20-40% selon configuration
- **Vitesse de traitement** : 100-1000+ textes/seconde
- **Qualité** : Préservation du sens, tokens cohérents
- **Flexibilité** : Adaptation facile aux besoins métier

## 🚀 **Utilisation en Production**

Votre pipeline est maintenant prêt pour :

```python
# Usage simple
from text_preprocessor import TextPreprocessor

# Initialisation
preprocessor = TextPreprocessor("ecommerce")

# Traitement
tokens = preprocessor.preprocess("Texte à traiter")
corpus_tokens = preprocessor.preprocess(["Texte 1", "Texte 2"])

# Analyse
stats = preprocessor.analyze(corpus)
```

## 🎓 **Prochaines Étapes**

1. **📊 Module 3** : Représentation vectorielle (TF-IDF, embeddings)
2. **🤖 Module 4** : Modèles de classification (Naive Bayes, SVM)
3. **🧠 Module 5** : Deep Learning (réseaux de neurones, LSTM)
4. **🤗 Module 6** : Transformers et modèles pré-entraînés

## 💡 **Conseils pour la Suite**

- **Testez** votre pipeline sur vos propres données
- **Mesurez** l'impact sur vos métriques business
- **Adaptez** les configurations selon vos besoins
- **Documentez** vos choix de preprocessing
- **Itérez** en fonction des résultats

---

**🎉 Bravo ! Vous êtes maintenant capable de construire des pipelines de preprocessing NLP robustes et performants !**

**📚 Notebook suivant :** `../module3/index.html` - Représentation Vectorielle du Texte